# Importing Libs

In [1]:
from IPython.display import clear_output
import pickle
import random

import cv2

from kotoamatsukami import HandDetector

# Set the Subject and Distance!

In [2]:
subject_name = "teste"

distance = "50cm"

# Gestures Info Handling

In [4]:
gestures_names = {

    0: "click",
    1: "closed",
    2: "down",
    3: "mouse_tracking",
    4: "negative_closed",
    5: "negative_mouse_tracking",
    6: "negative_side",
    7: "negative_up",
    8: "side",
    9: "up"

}

# gestures_features = {

#     0: [],
#     1: [],
#     2: [],
#     3: [],
#     4: [],
#     5: [],
#     6: [],
#     7: [],
#     8: [],
#     9: [],

# }

# Aux functions

In [5]:
def get_gestures_features(subject: str, distance: str, gest_names: dict) -> dict:
    gestures_features = dict()

    for gesture_id, name in gest_names.items():
        try:
            with open(f"data/randomized/{subject}/{distance}/{name}", "rb") as file:
                features = pickle.load(file)
        
        except:
            features = []
    
        gestures_features[gesture_id] = features
    
    return gestures_features

In [6]:
def collection_done(gest_dict: dict, num_features: int) -> bool:
    gest_features = list(gest_dict.values())

    if all(len(l) == num_features for l in gest_features):
        return True

    else:
        return False

In [7]:
def get_random_gesture(gest_dict: dict, num_features: int) -> int:
    possible_gestures = []

    for gesture_id, gest_features in gest_dict.items():
        if len(gest_features) < num_features:
            possible_gestures.append(gesture_id)
    
    if possible_gestures == []:
        return None

    random_gesture = random.choice(possible_gestures)

    return random_gesture

In [8]:
def print_percentage_done(gest_features: dict, num_features_per_gesture: int) -> None:
    clear_output(wait = True)

    total = len(gest_features.keys())*num_features_per_gesture

    done = sum([len(value) for value in gest_features.values()])

    percentage = round(done/total*100, 2)
    
    print(f"-------------   {percentage}%   -------------", flush = True)

In [9]:
def save_features(subject: str, distance: str, gest_dict: dict, gest_names: dict) -> None:
    for gesture_id, features in gest_dict.items():
        gesture_name = gest_names.get(gesture_id)

        with open(f"data/randomized/{subject}/{distance}/{gesture_name}", "wb") as file:
            pickle.dump(features, file)

# Main

In [10]:
gestures_features = get_gestures_features(subject = subject_name, distance = distance, gest_names = gestures_names)

num_frames_per_gesture = 50

In [12]:
stream   = cv2.VideoCapture(0)
detector = HandDetector(max_hands = 1, detection_con = 0.95)

HARD_STOP = False

while not collection_done(gest_dict = gestures_features, num_features = num_frames_per_gesture):
    print_percentage_done(gest_features = gestures_features, num_features_per_gesture = num_frames_per_gesture)

    if HARD_STOP:
        break

    gesture = get_random_gesture(gest_dict = gestures_features, num_features = num_frames_per_gesture)

    gesture_name  = gestures_names.get(gesture)
    gesture_image = cv2.imread(f"gestures/{gesture_name}.png", 0)
    
    while True:
        if cv2.waitKey(1) == ord("x"):
            break

        if cv2.waitKey(1) == ord("t"):
            HARD_STOP = True
            break
        
        _grabbed, raw_frame = stream.read()

        hands, image = detector.find_hands(image = raw_frame)

        if len(hands) != 0:
            features = hands[0]["landmarks"]

        cv2.imshow("Video", image)
        cv2.imshow("Gesture", gesture_image)

    gestures_features[gesture].append(features)

save_features(subject = subject_name, distance = distance, gest_dict = gestures_features, gest_names = gestures_names)

stream.release()
cv2.destroyAllWindows()
    

-------------   12.2%   -------------
